### Yolo Architecture (Architecture; 구성 요소, 동작 원리)

YOLO model 구현 (다른 방향에서 접근한 코드)

---
### Step 1. YOLOv5 Backbone 구현 (논문 기반)
- 이미지가 입력되었을 때 "특징 추출(feature extraction)" 하는 단계

In [1]:
# ============================================================
# 🧩 Step 1-1. 환경 세팅
# ============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", device)

✅ Device: cpu


In [2]:
# ============================================================
# 🧩 Step 1-2. Focus 모듈
# ============================================================

class Focus(nn.Module):
    """
    🔹 논문 기반 설명:
        - YOLOv5은 입력 이미지를 stride=2 Conv로 줄이지 않고,
          2x2 영역을 채널 방향으로 합쳐 정보 손실을 최소화.
        - 즉, (H, W) → (H/2, W/2) 대신, 채널(3→12)로 확장 후 Conv.

    🔹 이유:
        - 공간정보(spatial feature)를 유지하면서 연산 효율성을 높임.
        - 작은 물체의 위치 손실을 줄임.
    """
    def __init__(self, in_channels=3, out_channels=32, kernel_size=3):
        super().__init__()
        self.conv = nn.Conv2d(in_channels * 4, out_channels, kernel_size, stride=1, padding=1)

    def forward(self, x):
        # 4개 패치로 나눠 채널방향으로 결합
        patch = torch.cat([
            x[..., ::2, ::2],
            x[..., 1::2, ::2],
            x[..., ::2, 1::2],
            x[..., 1::2, 1::2]
        ], dim=1)
        return self.conv(patch)

# ✅ 테스트
x = torch.randn(1, 3, 640, 640)
focus = Focus()
y = focus(x)
print("출력 shape:", y.shape)  # (1, 32, 320, 320)

출력 shape: torch.Size([1, 32, 320, 320])


In [3]:
# ============================================================
# 🧩 Step 1-3. Conv 블록
# ============================================================

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

# ✅ 테스트
block = Conv(32, 64, 3, 2)
out = block(torch.randn(1, 32, 320, 320))
print("출력 shape:", out.shape)

출력 shape: torch.Size([1, 64, 160, 160])


In [4]:
# ============================================================
# 🧩 Step 1-4. Bottleneck 블록
# ============================================================

class Bottleneck(nn.Module):
    """
    🔹 내부 구조:
        1x1 Conv → 3x3 Conv (Residual 연결)
    🔹 이유:
        - 1x1 Conv로 채널 축소 후 3x3 Conv로 복원 → 연산 효율성
        - Skip 연결로 gradient 흐름 개선 (ResNet 아이디어)
    """
    def __init__(self, in_channels, out_channels, shortcut=True):
        super().__init__()
        self.conv1 = Conv(in_channels, out_channels, 1, 1)
        self.conv2 = Conv(out_channels, out_channels, 3, 1)
        self.use_add = shortcut and in_channels == out_channels

    def forward(self, x):
        y = self.conv2(self.conv1(x))
        return x + y if self.use_add else y

# ✅ 테스트
bottle = Bottleneck(64, 64)
out = bottle(torch.randn(1, 64, 160, 160))
print("출력 shape:", out.shape)

출력 shape: torch.Size([1, 64, 160, 160])


In [5]:
# ============================================================
# 🧩 Step 1-5. CSPBlock (Cross Stage Partial)
# ============================================================

class CSPBlock(nn.Module):
    """
    🔹 구조:
        입력 → 두 경로로 분할
          - 한쪽은 Bottleneck 반복
          - 다른 한쪽은 identity
        이후 concat → Conv → 출력
    🔹 이유:
        - 중복 gradient 흐름 방지 (feature redundancy 감소)
        - 성능 유지하면서 연산량 감소
    """
    def __init__(self, in_channels, out_channels, num_blocks=1):
        super().__init__()
        hidden = out_channels // 2
        self.conv1 = Conv(in_channels, hidden, 1, 1)
        self.conv2 = Conv(in_channels, hidden, 1, 1)
        self.blocks = nn.Sequential(*[Bottleneck(hidden, hidden) for _ in range(num_blocks)])
        self.conv3 = Conv(2 * hidden, out_channels, 1, 1)

    def forward(self, x):
        y1 = self.blocks(self.conv1(x))
        y2 = self.conv2(x)
        out = torch.cat((y1, y2), dim=1)
        return self.conv3(out)

# ✅ 테스트
csp = CSPBlock(64, 128, num_blocks=2)
out = csp(torch.randn(1, 64, 160, 160))
print("출력 shape:", out.shape)

출력 shape: torch.Size([1, 128, 160, 160])


In [6]:
# ============================================================
# 🧩 Step 1-6. YOLOv5 Backbone 구성 (multi-sclae 출력)
# ============================================================

class YOLOBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.focus = Focus(3, 32)
        self.conv1 = Conv(32, 64, 3, 2)
        self.csp1 = CSPBlock(64, 128, 3)
        self.csp2 = CSPBlock(128, 256, 9)
        self.csp3 = CSPBlock(256, 512, 9)

    def forward(self, x):
        x = self.focus(x)           # (32, 320, 320)
        x = self.conv1(x)           # (64, 160, 160)
        c1 = self.csp1(x)           # (128, 160, 160)
        c2 = self.csp2(c1)          # (256, 80, 80)
        c3 = self.csp3(c2)          # (512, 40, 40)
        return c1, c2, c3           # 3개 feature map 반환

### Step 2. YOLOv5 Neck + Head 구현
- Neck (PANet 구조)
  - Backbone 에서 추출한 다양한 크기의 feature map 을 연결
  - 작은 물체부터 큰 물체까지 다양한 스케일을 감지할 수 있도록 함
  - FPN + PANet 구조를 결합하여 정보가 위아래로 흐름

In [7]:
# ============================================================
# 🧩 Step 2-1. Neck (PANet 구조)
# ============================================================

class YOLOv5Neck(nn.Module):
    """
    🔹 논문 기반 설명:
        - YOLOv5의 Neck은 FPN(Feature Pyramid Network)과 PANet(Path Aggregation Network)의
          장점을 결합한 구조.
        - FPN: 상위 계층의 semantic feature를 하위로 전달.
        - PANet: 하위 계층의 localization feature를 상위로 전달.

    🔹 이유:
        - 서로 다른 스케일의 물체를 감지하기 위해 다중 feature map을 결합.
        - 작은 물체(고해상도 feature)와 큰 물체(저해상도 feature) 모두 처리 가능.
    """
    def __init__(self, channels=[128, 256, 512]):
        super().__init__()
        c1, c2, c3 = channels

        # FPN
        self.conv1 = Conv(c3, c2, 1, 1)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.csp_topdown = CSPBlock(c2 + c2, c2, num_blocks=3)

        # PANet
        self.conv2 = Conv(c2, c1, 1, 1)
        self.downsample = Conv(c1, c2, 3, 2)
        self.csp_bottomup = CSPBlock(c2 + c3, c3, num_blocks=3)

    def forward(self, x_small, x_medium, x_large):
        # FPN (Top-down)
        up = self.upsample(self.conv1(x_large))
        if up.shape != x_medium.shape:  # 자동 크기 보정
            up = F.interpolate(up, size=x_medium.shape[-2:], mode='nearest')
        merge1 = torch.cat([up, x_medium], dim=1)
        fpn_out = self.csp_topdown(merge1)

        # PANet (Bottom-up)
        down = self.downsample(self.conv2(fpn_out))
        if down.shape != x_large.shape:
            down = F.interpolate(down, size=x_large.shape[-2:], mode='nearest')
        merge2 = torch.cat([down, x_large], dim=1)
        pan_out = self.csp_bottomup(merge2)

        return fpn_out, pan_out

# ✅ 테스트
x_small = torch.randn(1, 128, 80, 80)
x_medium = torch.randn(1, 256, 40, 40)
x_large = torch.randn(1, 512, 20, 20)
neck = YOLOv5Neck()
f1, f2 = neck(x_small, x_medium, x_large)
print("FPN 출력:", f1.shape)
print("PAN 출력:", f2.shape)

FPN 출력: torch.Size([1, 256, 40, 40])
PAN 출력: torch.Size([1, 512, 20, 20])


Head
- Neck 에서 통합된 feature 를 입력받아 (x, y, w, h, confidence, class) 예측
- 각 스케일마다 anchor 기반 예측 수행

In [8]:
# ============================================================
# 🧩 Step 2-2. Detection Head
# ============================================================

class YOLOv5Head(nn.Module):
    """
    🔹 논문 기반 설명:
        - 각 스케일 feature에서 anchor box마다 (x, y, w, h, conf, class) 예측.
        - sigmoid 활성함수로 각 값을 0~1 사이로 정규화.

    🔹 이유:
        - x, y: 셀(grid) 내 상대 좌표 (sigmoid)
        - w, h: anchor box 비율 기반 스케일링 (exp or sigmoid)
        - conf: 물체 존재 확률
        - class: softmax 또는 sigmoid로 클래스 확률 예측
    """
    def __init__(self, channels=[128, 256, 512], num_classes=3, num_anchors=3):
        super().__init__()
        self.detect_layers = nn.ModuleList()
        for c in channels:
            self.detect_layers.append(
                nn.Conv2d(c, num_anchors * (5 + num_classes), 1)
            )

        self.num_classes = num_classes
        self.num_anchors = num_anchors

    def forward(self, features):
        """
        features: neck에서 나온 다중 스케일 feature map list
        """
        outputs = []
        for i, f in enumerate(features):
            pred = self.detect_layers[i](f)
            B, _, H, W = pred.shape
            pred = pred.view(B, self.num_anchors, 5 + self.num_classes, H, W)
            outputs.append(pred)
            print(f"스케일 {i+1} 출력:", pred.shape)
        return outputs

# ✅ 테스트
features = [torch.randn(1, 128, 80, 80),
            torch.randn(1, 256, 40, 40),
            torch.randn(1, 512, 20, 20)]
head = YOLOv5Head(num_classes=3)
outputs = head(features)

스케일 1 출력: torch.Size([1, 3, 8, 80, 80])
스케일 2 출력: torch.Size([1, 3, 8, 40, 40])
스케일 3 출력: torch.Size([1, 3, 8, 20, 20])


YOLOv5 전체 구조 결합 (Backbone + Neck + Head)
- 전체 구조를 연결해서 입력 -> feature 추출 -> multi-scale 예측까지 완성

In [9]:
# ============================================================
# 🧩 Step 2-3. YOLOv5 전체 모델 결합
# ============================================================

class YOLOv5Model(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.backbone = YOLOBackbone()
        self.neck = YOLOv5Neck()
        self.head = YOLOv5Head(num_classes=num_classes)

    def forward(self, x):
        # Backbone → 3개의 feature map 출력
        c1, c2, c3 = self.backbone(x)

        # Neck
        fpn_out, pan_out = self.neck(c1, c2, c3)

        # Head
        outputs = self.head([c1, fpn_out, pan_out])
        return outputs

# ✅ 테스트
x = torch.randn(1, 3, 640, 640)
model = YOLOv5Model(num_classes=3)
outputs = model(x)

스케일 1 출력: torch.Size([1, 3, 8, 160, 160])
스케일 2 출력: torch.Size([1, 3, 8, 160, 160])
스케일 3 출력: torch.Size([1, 3, 8, 160, 160])


---
3️⃣	학습 및 평가 파이프라인 구축	dataset loader, loss, optimizer, metrics	실제 데이터셋으로 훈련/검증 수행

4️⃣	결과 분석 및 시각화	Precision, Recall, mAP, confusion matrix	논문 수준의 결과 재현